In [1]:
"""
UNet configs comparison: from batchflow, from batchflow with fix, mine
"""

'\nUNet configs comparison: from batchflow, from batchflow with fix, mine\n'

In [2]:
# add weights in ce loss ?

In [3]:
import sys
sys.path.append("..")

In [4]:
import os
import shutil

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import batchflow
from batchflow import Pipeline, B, C, V, D
from batchflow.opensets import PascalSegmentation
from batchflow.research import Research, Option, Domain, Results, PrintLogger, RP, REP, KV
from batchflow.models.metrics import Loss
from batchflow.models.torch import UNet, EncoderDecoder

In [5]:
BATCH_SIZE = 64
ITERATIONS = 2000
# N_EPOCHS = 100
N_REPS = 10 
IMAGE_SHAPE = (160, 160)
GPU_ids = [3, 4, 6]
dataset = PascalSegmentation(bar='n')

In [6]:
def clear_previous_results(res_name):
    if os.path.exists(res_name):
        shutil.rmtree(res_name)

# Models configs

In [7]:
num_classes = 2

task_config = {
    'inputs/targets/classes': num_classes,  
    'head/layout': 'c',
    'head/filters': num_classes,
    'head/kernel_size': 1,
    'loss': 'ce',
    'optimizer': 'Adam'
}

# UNet from batchflow
UNet_bf = {'inputs': {'targets': {'classes': num_classes}},
 'placeholder_batch_size': 2,
 'device': None,
 'benchmark': True,
 'profile': False,
 'microbatch': None,
 'sync_frequency': 1,
 'optimizer': 'Adam',
 'decay': None,
 'amp': True,
 'sam_rho': 0.0,
 'sam_individual_norm': True,
 'order': ['initial_block', 'body', 'head'],
 'initial_block': {},
 'body': {'encoder': {'downsample': {'layout': 'p',
    'pool_size': 2,
    'pool_strides': 2},
   'num_stages': 4,
   'order': ['block', 'skip', 'downsampling'],
   'blocks': {'base': batchflow.models.torch.blocks.DefaultBlock,
    'layout': 'cna cna',
    'kernel_size': 3,
    'filters': [64, 128, 256, 512]}},
  'decoder': {'skip': True,
   'num_stages': None,
   'factor': None,
   'upsample': {'layout': 'tna'},
   'combine': {'op': 'concat', 'leading_index': 1},
   'order': ['upsampling', 'combine', 'block'],
   'blocks': {'base': batchflow.models.torch.blocks.DefaultBlock,
    'layout': 'cna cna',
    'kernel_size': 3,
    'filters': [512, 256, 128, 64]}},
  'embedding': {'base': batchflow.models.torch.blocks.DefaultBlock,
   'layout': 'cna cna',
   'kernel_size': 3,
   'filters': 1024}},
 'head': {'layout': 'c',
  'filters': num_classes,
  'kernel_size': 1,
  'target_shape': None,
  'classes': num_classes,
  'units': num_classes},
 'common': {'data_format': 'channels_first'},
 'predictions': None,
 'output': None,
 'loss': 'ce'}

# fixed UNet from batchflow
config_bf_with_fix = UNet_bf.copy()
config_bf_with_fix['body/decoder/upsample'] = dict(layout='tna', filters=[512, 256, 128, 64])

In [8]:
# my_UNet
downsample_depth = 4

my_config = {
    'body/encoder': {
        'num_stages': downsample_depth,
        'order': ['block', 'skip', 'downsampling']
    },    
    'body/encoder/blocks': {
        'layout': 'cna cna',
        'filters': [32*pow(2, i) for i in range(1, downsample_depth+1)]
    },
    'body/encoder/downsample': {
        'layout': 'p'
    },    
    
    'body/embedding': {
        'layout': 'cna cna', 
        'filters': [64*pow(2, downsample_depth), 64*pow(2, downsample_depth)]
    },   

    'body/decoder': {
        'num_stages': downsample_depth,
        'order': ['upsampling', 'combine', 'block']
    },
    'body/decoder/upsample': {
        'layout': 'tna',
        'filters': [32*pow(2, i) for i in range(downsample_depth, -1, -1)]
    },
    'body/decoder/combine': {
        'op': 'concat'
    },
    'body/decoder/blocks': {
        'layout': 'cna cna',
        'filters': [64*pow(2, i-1) for i in range(downsample_depth, -1, -1)]
    }
}
my_config.update(task_config)

# train

In [9]:
def process_mask(x):
    x = np.squeeze(x)
    np.place(x, x != 0, 1)
    return x

train_ppl = (dataset.train.p
    .init_variable('train_loss', [])
    .init_model('dynamic', C('model'), 'model', config=C('config'))
    .resize(size=IMAGE_SHAPE, src=['images', 'labels'], dst=['images', 'labels'])
    .to_array(channels='first', src=['images', 'labels'], dst=['images', 'labels'])
    .process_mask(B('labels'), save_to=B('labels'))
    .train_model('model', B('images'), B('labels'),
                fetches='loss', save_to=V('train_loss', mode='a'))
    .run_later(BATCH_SIZE, shuffle=True, n_epochs=None)
)

In [10]:
configs = [KV(task_config, "config_bf"), KV(config_bf_with_fix, "config_bf_with_fix"), KV(my_config, "my_config")]
domain = Option('model', [UNet, UNet, EncoderDecoder]) @ Option('config', configs)
list(domain.iterator)

# configs = [KV(my_config, "my_config")]
# domain = Option('model', [EncoderDecoder]) @ Option('config', configs)
# list(domain.iterator)

[ConfigAlias({'model': 'UNet', 'config': 'config_bf', 'repetition': '0'}),
 ConfigAlias({'model': 'UNet', 'config': 'config_bf_with_fix', 'repetition': '0'}),
 ConfigAlias({'model': 'EncoderDecoder', 'config': 'my_config', 'repetition': '0'})]

# performance

In [11]:
test_ppl = (dataset.test.p
                .import_model('model', C('import_from'))
                .init_variable('metrics')
                .init_variable('predictions')
                .resize(size=IMAGE_SHAPE, src=['images', 'labels'], dst=['images', 'labels'])
                .to_array(channels='first', src=['images', 'labels'], dst=['images', 'labels'])
                .process_mask(B('labels'), save_to=B('labels'))                
                .predict_model('model', B('images'), fetches='predictions',
                               save_to=V('predictions'))
                .gather_metrics('classification', B('labels'), V('predictions'),
                                          axis=1, fmt='logits', num_classes=num_classes,
                                          save_to=V('metrics', mode='update'))
                .run_later(BATCH_SIZE, shuffle=False, n_epochs=1)
            )

In [12]:
metrics = ['acc', 'precision', 'recall', 'f1_score', 'iou']

In [13]:
TEST_EXECUTE_FREQ = ['#0', 100, 'last']

res_name = 'UNet_pascal_train_test_research'
clear_previous_results(res_name)

research = (Research()
#             .add_logger('print')
            .init_domain(domain, n_reps=N_REPS)
            .add_pipeline(train_ppl, variables='train_loss', name='train_ppl')
            .add_pipeline(test_ppl, name='test_ppl',
                         execute=TEST_EXECUTE_FREQ, run=True, import_from=RP('train_ppl'))
            .get_metrics(pipeline='test_ppl', metrics_var='metrics', metrics_name=metrics,
                         returns=metrics, execute=TEST_EXECUTE_FREQ)
           )



research.run(n_iters=ITERATIONS, name=res_name, bar=True, workers=len(GPU_ids), devices=GPU_ids)

Research UNet_pascal_train_test_research is starting...


Domain updated: 0: 100%|██████████| 60000/60000.0 [3:31:31<00:00,  4.73it/s]  


In [14]:
df = research.load_results().df

In [15]:
df.loc[df.name=='test_ppl_metrics'].sort_values('acc', ascending=False).head()

,name,train_loss,acc,precision,recall,f1_score,iou,iteration,sample_index,model,config,repetition,update
42438,test_ppl_metrics,NaN,0.825632,0.730071,0.673370,0.656126,0.535470,1799,1885998657,EncoderDecoder,my_config,5,0
46482,test_ppl_metrics,NaN,0.825386,0.721986,0.711151,0.674168,0.553418,1999,3809067587,EncoderDecoder,my_config,9,0
10103,test_ppl_metrics,NaN,0.825208,0.727242,0.678637,0.658885,0.539216,1899,2641868915,UNet,config_bf_with_fix,6,0
54564,test_ppl_metrics,NaN,0.825164,0.729920,0.684688,0.662428,0.542888,1799,3550521261,EncoderDecoder,my_config,4,0
10102,test_ppl_metrics,NaN,0.825146,0.723299,0.682841,0.658199,0.539319,1799,2641868915,UNet,config_bf_with_fix,6,0


In [16]:
tmp = df.loc[df.name=='test_ppl_metrics']

In [17]:
tmp.groupby('config')[metrics].describe().T

config            config_bf  config_bf_with_fix   my_config
acc       count  210.000000          210.000000  210.000000
          mean     0.792276            0.787638    0.788398
          std      0.033142            0.056890    0.057804
          min      0.672622            0.551783    0.551740
          25%      0.788735            0.784229    0.788650
          50%      0.803004            0.805507    0.807736
          75%      0.813160            0.815338    0.815964
          max      0.820794            0.825208    0.825632
precision count  210.000000          210.000000  210.000000
          mean     0.680459            0.676628    0.679070
          std      0.053547            0.077294    0.079162
          min      0.544794            0.385173    0.385153
          25%      0.658534            0.660463    0.667602
          50%      0.694091            0.697892    0.705677
          75%      0.716729            0.717988    0.721052
          max      0.798017            0.787118    0.785630
recall    count  210.000000          210.000000  210.000000
          mean     0.655094            0.670486    0.666193
          std      0.136322            0.071129    0.074490
          min      0.150919            0.346912    0.316962
          25%      0.629591            0.640327    0.628542
          50%      0.684051            0.682205    0.677849
          75%      0.724626            0.707125    0.706131
          max      0.890985            0.829081    0.823114
f1_score  count  210.000000          210.000000  210.000000
          mean     0.604654            0.613200    0.612433
          std      0.101087            0.059740    0.060091
          min      0.182663            0.404457    0.360283
          25%      0.602137            0.599713    0.600471
          50%      0.634394            0.630670    0.629365
          75%      0.650915            0.653389    0.651447
          max      0.676005            0.682253    0.678267
iou       count  210.000000          210.000000  210.000000
          mean     0.482081            0.488123    0.487075
          std      0.090281            0.061225    0.061659
          min      0.114363            0.293184    0.253794
          25%      0.472300            0.466214    0.472403
          50%      0.509871            0.506207    0.504733
          75%      0.527349            0.530550    0.528748
          max      0.555974            0.563324    0.558338